### Import module

In [1]:
from glob import glob
import os, sys
from os.path import join, dirname

import datetime, time
from datetime import datetime
from dateutil.relativedelta import relativedelta

import csv
from glob import glob
import chardet
import numpy as np
import pandas as pd

from tqdm import notebook

In [2]:
root_dir = '../Data'

EMA_dir = root_dir + '/KOR_EMA_Data'
EMA_folder_list = glob(EMA_dir + '/KOR*')
preprocessed_dir = root_dir + '/Preprocessed'

In [3]:
## EMA Data collumn name
EMA_col_name = ["Start Date","End Date","Time Scheduled","Duration (in seconds)","Finished","Participant ID","Location - Lat",
                "Location - Long","Survey","Question 1","Question 2","Question 3","Question 4.1","Question 4.2","Question 4.3",
                "Question 4.4","Question 4.5","Question 4.6","Question 4.7","Question 4.8","Question 5","Question 6","Question 7"]

In [4]:
for path in EMA_folder_list:
    
    TimeBased_file = path+'\\TimeBased.csv'
    TimeBased_data = pd.read_csv(TimeBased_file, header=3, encoding='utf-8')
    TimeBased_data = TimeBased_data[4:]
    col = TimeBased_data.columns.to_numpy()
    col = col[[0,1,2,3,4,5,6,7,8,9,19,20,10,11,12,13,14,15,16,17,18,21]]             ### Align to Eventbase column order
    TimeBased_data = TimeBased_data[col]
    TimeBased_data = TimeBased_data.to_numpy()
    
    Eventbased_file = path+'\\Eventbased.csv'
    Eventbased_data = pd.read_csv(Eventbased_file, header=3, encoding='utf-8' )
    Eventbased_data = Eventbased_data[4:]
    Eventbased_data = Eventbased_data.to_numpy()

    file_name = path.split('\\')[1] + '.csv'
    file_name = EMA_dir +'/'+ file_name
    
    print(file_name)
    with open(file_name, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Start Date', 'End Date','Time Scheduled','Duration (in seconds)','Finished','Participant ID',
            'Location - Lat', 'Location - Long', 'Survey', 'Question 1', 'Question 2', 'Question 3', 'Question 4.1', 
            'Question 4.2', 'Question 4.3', 'Question 4.4', 'Question 4.5', 'Question 4.6', 'Question 4.7',
            'Question 4.8', 'Question 5', 'Question 6', 'Question 7'])
        
        for i in range(len(TimeBased_data)):
            writer.writerow([TimeBased_data[i][0], TimeBased_data[i][1], TimeBased_data[i][2], TimeBased_data[i][3], TimeBased_data[i][4],
                           TimeBased_data[i][5], TimeBased_data[i][6], TimeBased_data[i][7], TimeBased_data[i][8], TimeBased_data[i][9],
                           TimeBased_data[i][10], TimeBased_data[i][11], TimeBased_data[i][12], TimeBased_data[i][13], TimeBased_data[i][14],
                           TimeBased_data[i][15], TimeBased_data[i][16], TimeBased_data[i][17], 'nan', TimeBased_data[i][18], 
                            TimeBased_data[i][19], TimeBased_data[i][20], TimeBased_data[i][21]])

        for j in range(len(Eventbased_data)):
            writer.writerow(Eventbased_data[j])

../Data/KOR_EMA_Data/KORp10_EMA_data.csv
../Data/KOR_EMA_Data/KORp11_EMA_data.csv
../Data/KOR_EMA_Data/KORp12_EMA_data.csv
../Data/KOR_EMA_Data/KORp13_EMA_data.csv
../Data/KOR_EMA_Data/KORp14_EMA_data.csv
../Data/KOR_EMA_Data/KORp15_EMA_data.csv
../Data/KOR_EMA_Data/KORp16_EMA_data.csv
../Data/KOR_EMA_Data/KORp17_EMA_data.csv
../Data/KOR_EMA_Data/KORp5_EMA_data.csv
../Data/KOR_EMA_Data/KORp6_EMA_data.csv
../Data/KOR_EMA_Data/KORp7_EMA_data.csv
../Data/KOR_EMA_Data/KORp8_EMA_data.csv
../Data/KOR_EMA_Data/KORp9_EMA_data.csv


In [4]:
EMA_data_list = glob(EMA_dir + '/*.csv')
target_data_list = []

for tmp in EMA_data_list:
    temp_name = tmp.split('\\')[1].split('_')[0]
    temp_name = preprocessed_dir + '/target_' + temp_name + '.csv'
    target_data_list.append(temp_name)

In [5]:
def convert24(time_stamp): 
    
    time_stamp = str(pd.to_datetime(time_stamp, infer_datetime_format=True))
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    
    if time_stamp[-2:] == "AM" or time_stamp[-2:] == "PM":
        if time_stamp[-2:] == "AM" and time_stamp[11:13] == "12": 
            return time_stamp[:11] + "00" + time_stamp[13:-2] 

        # remove the AM  
        elif time_stamp[-2:] == "AM": 
            return time_stamp[:-2] 

        # Checking if last two elements of time 
        # is PM and first two elements are 12 
        elif time_stamp[-2:] == "PM" and time_stamp[11:13] == "12": 
            return time_stamp[:-2] 

        else: 

            # add 12 to hours and remove PM 
            return time_stamp[:11] + str(int(time_stamp[11:13]) + 12) + time_stamp[13:-2]
    
    else: 
        return time_stamp

In [12]:
## Dictionary for response types
res = {"0" : "Playing",
       "1" : "Talking",
       "2" : "Petting",
       "3" : "TV / Radio",
       "4" : "Eating / Cooking",
       "5" : "Moved It",
       "6" : "None of the above",
       "7" : "Other"}

for x in range(len(EMA_data_list)):
    
    EMA = pd.read_csv(EMA_data_list[x], encoding='utf-8')
    ema_time = EMA['Start Date']
    for i in range(len(ema_time)):
        ema_time[i] = convert24(ema_time[i])
    EMA['Start Date'] = ema_time
    EMA = EMA.sort_values(['Start Date', 'End Date','Time Scheduled','Duration (in seconds)','Finished','Participant ID',
            'Location - Lat', 'Location - Long', 'Survey', 'Question 1', 'Question 2', 'Question 3', 'Question 4.1', 
            'Question 4.2', 'Question 4.3', 'Question 4.4', 'Question 4.5', 'Question 4.6', 'Question 4.7',
            'Question 4.8', 'Question 5', 'Question 6', 'Question 7'], ascending=True)
    EMA = EMA.to_records(index=False)
    
#     print(EMA)
#     break
    
    ## Row number in will-be-created file
    rowid = 1
    prox = 0
    
    ## Write a csv file
    print(target_data_list[x])
    with open(target_data_list[x], 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["RowID", "Modality_cat", "Modality", "Proximity", "Start Window", "End Window"])


        for i in range(len(EMA)):
            if EMA[i]["Question 1"] == "Yes":  # Check if the participant has interacted

                interation_type = []
                prox = 0  # prox reset
                
                ## Check the number of interaction types
                for j in range(12, 20):
                    if EMA[i][j] == "Yes":
                        interation_type.append(j-12)
                
                for k in range(len(interation_type)):
                    end_time = convert24(EMA[i]["Start Date"])
                    start_time = pd.to_datetime(end_time, infer_datetime_format=True) - pd.Timedelta(pd.offsets.Minute(15))
                    writer.writerow([rowid, interation_type[k], res[str(interation_type[k])], EMA[i]["Question 6"], start_time, end_time]) # Write data one (row) by one..
                    rowid += 1

<ipython-input-12-361dd1eea9ac>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ema_time[i] = convert24(ema_time[i])


../Data/Preprocessed/target_KORp10.csv
../Data/Preprocessed/target_KORp11.csv
../Data/Preprocessed/target_KORp12.csv
../Data/Preprocessed/target_KORp13.csv
../Data/Preprocessed/target_KORp14.csv
../Data/Preprocessed/target_KORp15.csv
../Data/Preprocessed/target_KORp16.csv
../Data/Preprocessed/target_KORp17.csv
../Data/Preprocessed/target_KORp5.csv
../Data/Preprocessed/target_KORp6.csv
../Data/Preprocessed/target_KORp7.csv
../Data/Preprocessed/target_KORp8.csv
../Data/Preprocessed/target_KORp9.csv
